# Generate the phenotypes for the hearing impairment traits from the UKBB for the 500K individuals with imputed data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# 1. Read in the data

## 1.1. Read in database

In [ ]:
# collect the necessary column names of the database for our analysis

with open("/mnt/mfs/statgen/UKBiobank/phenotype_files/HI_UKBB/092821_UKBB_486416ind_call90.csv") as fp:
    line = fp.readline() # header
    header = line.split(",")
    
    indiv = ["IID", "FID"]
    icd10_colnames = [col.strip('"') for col in header if "f.41270." in col]
    icd10_ages = [col.strip('"') for col in header if "f.41280." in col]
    icd9_colnames = [col.strip('"') for col in header if "f.41271." in col]
    icd9_ages = [col.strip('"') for col in header if "f.41281." in col]
    f20002_colnames = [col.strip('"') for col in header if "f.20002." in col]
    reported_sex = ["f.31.0.0"]
    genetic_sex = ["f.22001.0.0"]
    ethnicity = [col.strip('"') for col in header if "f.21000." in col]
    hearing_imp_f3393 = [col.strip('"') for col in header if "f.3393." in col]
    hearing_imp_f2247 = [col.strip('"') for col in header if "f.2247." in col]
    hearing_imp_f2257 = [col.strip('"') for col in header if "f.2257." in col]
    tin_cols = [col.strip('"') for col in header if "f.4803." in col]
    ages_f21003_col = [col.strip('"') for col in header if "f.21003." in col]
    ages_f131258_col = [col.strip('"') for col in header if 'f.131258.' in col]
    year_of_birth = [col.strip('"') for col in header if "f.34." in col]
    month_of_birth = [col.strip('"') for col in header if "f.52." in col]

In [ ]:
# combine the column names into one list
combined_cols = indiv + icd10_colnames + icd10_ages + icd9_colnames + icd9_ages + f20002_colnames + ethnicity + reported_sex + genetic_sex + hearing_imp_f3393 + hearing_imp_f2247 + hearing_imp_f2257 + tin_cols + ages_f21003_col + ages_f131258_col + year_of_birth + month_of_birth

In [ ]:
print(datetime.now())

In [ ]:
# database of all individuals that we are working with and the selected phenotypes
df = pd.read_csv("/mnt/mfs/statgen/UKBiobank/phenotype_files/HI_UKBB/092821_UKBB_486416ind_call90.csv", quotechar = '"', dtype="string", usecols=combined_cols)
df

In [ ]:
print(datetime.now())

## 1.2. Read in exclusion criteria for icd10, icd9, and self-report

In [ ]:
# csv file that contains information on the exclusion criteria for cases and controls
exclusion = pd.read_csv("/mnt/mfs/statgen/UKBiobank/phenotype_files/HI_UKBB/ICD10_9_selfreport_incl_excl.csv")
exclusion

## 1.3. Read in individuals in genotype array QC

In [ ]:
qc_individuals = pd.read_csv("/mnt/mfs/statgen/UKBiobank/results/092821_PCA_results_500K/white_europeans/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.white_europeans.filtered.fam", sep="\t", header=None)
qc_individuals

## 1.4. Read in PCA outlier file

In [ ]:
# outlier individuals that will need to be removed
outlier = pd.read_csv("/mnt/mfs/statgen/UKBiobank/results/092821_PCA_results_500K/092821_PCA_related_pval0.005/ukb47922_white_460649ind.092821_PCA_related_pval0.005.pca.projected.outliers", sep="\t", header=None)
outlier

# 2. Sample QC

## 2.1. Remove individuals that do not match for reported and genetic sex

In [ ]:
reported_sex = ["f.31.0.0"]
genetic_sex = ["f.22001.0.0"]

In [ ]:
# returns true only if 
def inconsistent_sexes(row):
    return pd.isna(row[genetic_sex[0]]) or row[reported_sex[0]] != row[genetic_sex[0]]

In [ ]:
# exclusion based on inconsistent sex
ex_sex = df[reported_sex + genetic_sex].apply(inconsistent_sexes, axis=1)

In [ ]:
filtered = df[~ex_sex]

In [ ]:
print(sum(ex_sex), "individuals removed because of inconsistency with the genetic and reported sex variables")

In [ ]:
print("Of these individuals", sum([1 for x in df[genetic_sex[0]].to_list() if pd.isna(x)]), "were NA for the genetic sex variable")

In [ ]:
filtered

## 2.2. Remove non-white individuals

In [ ]:
# set of answers for the ethnicity question
set(filtered[ethnicity[0]].to_list()).union( set(filtered[ethnicity[1]].to_list()) , set(filtered[ethnicity[2]].to_list()))

In [ ]:
# these should align with all possible options for ethnicity answers except for <NA>, Do not know, and Prefer not to answer
white = ['British', 'Irish', 'White','Any other white background']
african = ['Caribbean','White and Black Caribbean', 'African', 'White and Black African', 'Black or Black British', 'Any other Black background' ]
asian = ['Indian', 'Pakistani', 'White and Asian', 'Any other Asian background', 'Bangladeshi', 'Asian or Asian British']
mixed = ['Mixed', 'Any other mixed background']
chinese = ['Chinese']
other = ['Other ethnic group']

# figure out the ancestry of each individual
def ancestry(row):
    temp = [x for x in row[ethnicity] if not pd.isna(x) and x != "Prefer not to answer" and x != "Do not know"]
    if len(temp) == 0:
        return "Unknown"
    
    if len(set(temp)) == 1 and temp[0] in white: # if we have only one unique answer and the answer is in the white variable
        return "_".join(temp[0].split(" ")) # return the unique answer
    
    if len([x for x in temp if x in white]) == len(temp):
        return "Inconsistent_white"
    if len([x for x in temp if x in african]) == len(temp):
        return "African"
    if len([x for x in temp if x in asian]) == len(temp):
        return "Asian"
    if len([x for x in temp if x in mixed]) == len(temp):
        return "Mixed"
    if len([x for x in temp if x in chinese]) == len(temp):
        return "Chinese"
    if len([x for x in temp if x in other]) == len(temp):
        return "Other"
    return "Inconsistent"

Set an ancestry row that combines the ancestry answers from the database into one 

In [ ]:
filtered["ethnicity"] = filtered[ethnicity].apply(ancestry, axis=1)

In [ ]:
def find_non_white(row):
    return row["ethnicity"] not in white and row["ethnicity"] != "Unknown" and row["ethnicity"] != "Inconsistent_white" and row["ethnicity"] != "Any_other_white_background"

In [ ]:
ex_non_white = filtered[["ethnicity"]].apply(find_non_white, axis=1)

In [ ]:
filtered = filtered[~ex_non_white]

In [ ]:
print(sum(ex_non_white), "individuals removed for being non-white")

In [ ]:
filtered

In [ ]:
filtered[filtered["ethnicity"] == "British"]

In [ ]:
filtered[filtered["ethnicity"] == "Irish"]

In [ ]:
filtered[filtered["ethnicity"] == "White"]

In [ ]:
filtered[filtered["ethnicity"] == "Inconsistent_white"]

In [ ]:
filtered[filtered["ethnicity"] == "Unknown"]

In [ ]:
filtered[filtered["ethnicity"] == "Any_other_white_background"]

In [ ]:
saved_for_number_checking = filtered

In [ ]:
filtered = saved_for_number_checking

In [ ]:
filtered[["FID","IID","ethnicity"]].to_csv("/mnt/mfs/statgen/UKBiobank/phenotype_files/HI_UKBB/ukb47922_white_460649ind.pheno", sep="\t", index=False)

## 2.3. Only keep individuals that passed genotype array QC

In [ ]:
filtered

In [ ]:
qc_list = [str(i) for i in qc_individuals[0].to_list()]
def matches_qc_individuals(row):
    return row["FID"] in qc_list
filtered = filtered[filtered[["FID"]].apply(matches_qc_individuals, axis=1)]

# 2. Remove PCA outliers from the full database

Remove the outlier individuals from the full database if there exists any.

In [ ]:
# since the IID from the dataframe is in string the outlier ids have to be made into string as well
out_ids = [str(x) for x in outlier[0].to_list()] 

def find_outliers(row):
    return row["IID"] in out_ids

In [ ]:
ex_pca_outliers = filtered[["IID", "FID"]].apply(find_outliers, axis=1)

In [ ]:
filtered = filtered[~ex_pca_outliers]

In [ ]:
print(sum(ex_pca_outliers), "individuals removed for being pca outliers")

In [ ]:
#len(missing_cases - set(df["IID"].to_list()))

In [ ]:
filtered

# 3. Filter out exclusions from the full database

If individuals have certain codes from ICD 10, ICD 9, and self-reports they must be fully removed from the analysis. 

In [ ]:
# returns if the current individual should be excluded based on the exclusion list
def contains_exclusion(row, exclusion_list):
    for i in row:
        if not pd.isna(i) and i in exclusion_list:
            return True
            
    return False

## 3.1. Filter out ICD 10 exclusions

In [ ]:
# these are the columns that represent the icd10 columns in the database
icd10_colnames = [col for col in filtered if "f.41270" in col]

In [ ]:
# get a dataframe that only contains the icd10 columns from the full database
icd10 = filtered[icd10_colnames]
icd10

In [ ]:
# get rows from exclusion database that contian the codes that need to be removed for icd10
exclude_icd10 = exclusion[(exclusion["UKBB_field_code"] == "f.41270") & (exclusion["Excluded_fulldb_lateonsetHI"] == 'Y') ]
exclude_icd10

In [ ]:
# get the icd10 codes that should be excluded from database
ex_critia_icd10 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_icd10["Phenotype"].tolist()]
ex_critia_icd10

In [ ]:
# collect the individuals that should be excluded because of icd10
ex_fxn_icd10 = lambda row: contains_exclusion(row, ex_critia_icd10)
ex_10 = icd10.apply(ex_fxn_icd10, axis=1)

In [ ]:
# remove them from the working database (which is now filtered. df remains unchanged)
filtered = filtered[~ex_10]

In [ ]:
print(sum(ex_10), "individuals removed because of icd10 codes")

In [ ]:
filtered

In [ ]:
#len(missing_cases - set(filtered["IID"].to_list()))

## 3.2. Filter out ICD 9 exclusions

In [ ]:
# these are the columns that represent the icd9 columns in the working database
icd9_colnames = [col for col in filtered if "f.41271" in col]

In [ ]:
# get a dataframe that only contains the icd9 columns from the working database
icd9 = filtered[icd9_colnames]
icd9

In [ ]:
# get rows from exclusion database that contian the codes that need to be removed for icd9
exclude_icd9 = exclusion[(exclusion["UKBB_field_code"] == "f.41271") & (exclusion["Excluded_fulldb_lateonsetHI"] == 'Y') ]
exclude_icd9

In [ ]:
# get the icd9 codes that should be excluded from the working database
ex_critia_icd9 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_icd9["Phenotype"].tolist()]
ex_critia_icd9

In [ ]:
# collect the individuals that should be excluded because of icd9
ex_fxn_icd9 = lambda row: contains_exclusion(row, ex_critia_icd9)
ex_9 = icd9.apply(ex_fxn_icd9, axis=1)

In [ ]:
# remove them from the working database
filtered = filtered[~ex_9]

In [ ]:
print(sum(ex_9), "individuals removed because of icd9 codes")

In [ ]:
filtered

In [ ]:
#len(missing_cases - set(filtered["IID"].to_list()))

## 3.3. Filter out f.20002 exclusions

In [ ]:
# these are the columns that represent the self-report columns in the working database
f20002_colnames = [col for col in filtered if "f.20002" in col]

In [ ]:
# get a dataframe that only contains the self-report columns from the working database
f20002 = filtered[f20002_colnames]
f20002

In [ ]:
# get rows from exclusion database that contian the codes that need to be removed for self-report
exclude_f20002 = exclusion[(exclusion["UKBB_field_code"] == "f.20002") & (exclusion["Excluded_fulldb_lateonsetHI"] == 'Y') ]
exclude_f20002

In [ ]:
# get the self-report codes that should be excluded from the working database
ex_critia_f20002 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_f20002["Phenotype"].tolist()]
ex_critia_f20002

In [ ]:
# collect the individuals that should be excluded because of self-report
ex_fxn_f20002 = lambda row: contains_exclusion(row, ex_critia_f20002)
ex_f20002 = f20002.apply(ex_fxn_f20002, axis=1)

In [ ]:
# remove them from the working database
filtered = filtered[~ex_f20002]

In [ ]:
print(sum(ex_f20002), "individuals removed because of self-reported codes")

In [ ]:
filtered

In [ ]:
#len(missing_cases - set(filtered["IID"].to_list()))

# 4. Identify Sex Column

In [ ]:
# male is denoted a 0, female as 1
def find_sex(row):
    if row["f.31.0.0"] == "Male":
        return 0
    return 1

sex = filtered[["f.31.0.0"]].apply(find_sex, axis=1)
sex

In [ ]:
filtered["sex"] = sex

In [ ]:
filtered

In [ ]:
filtered = filtered.reset_index()

In [ ]:
filtered

In [ ]:
saved_filtered = filtered

In [ ]:
filtered = saved_filtered

In [ ]:
filtered[filtered["ethnicity"] == "British"]

In [ ]:
filtered[filtered["ethnicity"] == "Irish"]

In [ ]:
filtered[filtered["ethnicity"] == "White"]

In [ ]:
filtered[filtered["ethnicity"] == "Inconsistent_white"]

In [ ]:
filtered[filtered["ethnicity"] == "Unknown"]

In [ ]:
filtered[filtered["ethnicity"] == "Any_other_white_background"]

# 5. f.3393, f.2247, f.2257, and Mendelian

## 5.1. Remove inconsistencies or unclear individuals

Some individuals might be unclear on if they do or do not have hearing difficulties or are inconsistent (found in f.3393, f.2247, and f.2257), in which case they cannot be considered either controls or cases and must be removed.

The conditions for being removed are as follows:
* Saying I don't know after saying either yes or no
* Only saying I don't know or prefer not to say
* Being completely deaf

### 5.1.1. Prior to filtering for inconsistencies

<b>Hearing difficulty/problems with background noise</b> <br>
f.2257 = {'Yes': 81218, NA : 513774, 'No': 131091, 'Do not know': 4409, 'Prefer not to answer': 208}

<b>Hearing difficult/problems</b><br>
f.2247 = {'No': 151758, : 513806, 'Yes': 55437, 'Do not know': 9489, 'Prefer not to answer': 171, 'I am completely deaf': 39}

<b>Hearing aid user</b><br>
f.3393 = {'No': 145486, : 577795, 'Yes': 7237, 'Prefer not to answer': 182}

### 5.1.2. Setup for inconsistency filtering

In [ ]:
# collect all the columns 
# redefining here for clarity

hearing_imp_f3393 = [col for col in filtered if "f.3393" in col]
hearing_imp_f2247 = [col for col in filtered if "f.2247" in col]
hearing_imp_f2257 = [col for col in filtered if "f.2257" in col]

icd_10_cols = [col for col in df if "f.41270" in col]
icd_9_cols = [col for col in df if "f.41271" in col]

In [ ]:
# for hearing impairement questions we code the answers with the following for comparison
hearing_ans = {"Do not know":9, "Yes":1, "No":0}

# will contain the complete set of actual combinations of answers from individuals in the database
options = set()

# pass one pheno at a time to this function
def find_options(row):
    answer = "".join([str(hearing_ans[i]) for i in row if not pd.isna(i) and i != "I am completely deaf" and i != "Prefer not to answer"])
    options.add(answer)

In [ ]:
# this builds the options set to contain a set of all the unique answers the individuals in the database have had
# over their hearing impairment questions
hearing_imp_qs = filtered[hearing_imp_f3393]
s = hearing_imp_qs.apply(find_options, axis=1)
hearing_imp_qs = filtered[hearing_imp_f2247]
s = hearing_imp_qs.apply(find_options, axis=1)
hearing_imp_qs = filtered[hearing_imp_f2257]
s = hearing_imp_qs.apply(find_options, axis=1)

In [ ]:
options

In [ ]:
# we might have inconsistencies if we have don't knows mixed with other answers or even yes and no together
# options contains the set of all unique codes 

do_not_know_no = [i for i in options if '0' in i and '9' in i and '1' not in i]
do_not_know_yes = [i for i in options if '0' not in i and '9' in i and '1' in i]
yes_no = [i for i in options if '0' in i and '9' not in i and '1' in i]
with_all_three = [i for i in options if '0' in i and '9' in i and '1' in i]

# collecting all possible occurances that can be flagged for being inconsistent
might_inconsistent = do_not_know_no + do_not_know_yes + yes_no + with_all_three

# these are exceptions to the possibilities we have that can be flagged for being inconsistent
exceptions = ["91","911","9111","991","0001","001","0011","01","011","0111", "0091", "091","0911","0991","9001","901","9011", "90", "900", "9000", "990"]

# collect the list of answer combinations that are actually inconsistent by removingthe answer combinations that are exceptions
inconsistent = [i for i in might_inconsistent if i not in exceptions]

In [ ]:
inconsistent

### 5.1.3. Filtering out the data

In [ ]:
# return True if you find an individual that has NA for every answer in each column
def find_empty(row):
    for i in row:
        if not pd.isna(i):
            return False
    return True

In [ ]:
# return True if an individual only answers Do not know but never Yes or No
def find_dont_know(row):
    temp = []
    for i in row:
        if not pd.isna(i):
            temp.append(i)
        
    if "Do not know" in temp and "Yes" not in temp and "No" not in temp:
        return True
    return False

In [ ]:
# will return true if that row should be removed
# pass one pheno at a time through the function
def find_inconsistencies(row):
    for i in row:
        if not pd.isna(i) and i == "I am completely deaf":
            return True
    
    answer = "".join([str(hearing_ans[i]) for i in row if not pd.isna(i) and i != "I am completely deaf" and i != "Prefer not to answer"])
    if answer in inconsistent:
        return True
    return False

# if we have individuals that either don't answer or prefer not to say only then we cancel them out
# return true if all the rows have no definitive answers
def find_all_none(row):
    for i in row:
        if not pd.isna(i) and (i == "Yes" or i == "No"):
            return False
    return True

In [ ]:
filtered

In [ ]:
# filter out inconsistencies for f3393
hearing_imp_qs = filtered[hearing_imp_f3393]
exclude = hearing_imp_qs.apply(find_inconsistencies, axis=1)
filtered = filtered[~exclude]

In [ ]:
len(missing_cases - set(filtered["IID"].to_list()))

In [ ]:
# filter out inconsistencies for f2247
hearing_imp_qs = filtered[hearing_imp_f2247]
exclude = hearing_imp_qs.apply(find_inconsistencies, axis=1)
filtered = filtered[~exclude]

In [ ]:
len(missing_cases - set(filtered["IID"].to_list()))

In [ ]:
# filter out inconsistencies for f2257
hearing_imp_qs = filtered[hearing_imp_f2257]
exclude = hearing_imp_qs.apply(find_inconsistencies, axis=1)
filtered = filtered[~exclude]

In [ ]:
len(missing_cases - set(filtered["IID"].to_list()))

In [ ]:
# filter out individuals that don't have a definiteive answer for any hearing aid questions
hearing_imp_qs = filtered[hearing_imp_f3393 + hearing_imp_f2247 + hearing_imp_f2257]
exclude = hearing_imp_qs.apply(find_all_none, axis=1)
filtered = filtered[~exclude]

In [ ]:
len(missing_cases - set(filtered["IID"].to_list()))

In [ ]:
filtered

In [ ]:
saved_2_filtered = filtered

In [ ]:
filtered = saved_2_filtered

## 5.2. Identify Pure Controls

Need to make sure that for f.3393, f.2247, and f.2257 we obtain the individuals that say no.

The conditions for saying no are as follows:
* Never saying yes
* Not being inconsistent
* Saying no at least once (prefer not to say is allowed)

We are also not including individuals to be part of the control group if they have certain codes for ICD9, ICD10, or f.20002 (this is in the case that they say no to all f.3393, f.2247, and f.2257). However these individuals can still be part of the cases

In [ ]:
# returns 0 if it's a ctrl or else 1
def find_ctrl(row):
    answer = "".join([str(hearing_ans[i]) for i in row if not pd.isna(i) and i != "Prefer not to answer"])
    if "0" in answer and "1" not in answer:
        return 0
    return 1

# returns 0 if it's a ctrl or else 1
# this is specific for f3393 because f3393 was only asked under certain circumstances
def find_ctrl_or_NA(row):
    for i in row:
        if not pd.isna(i) and i != "No" and i != "Prefer not to answer": # if we have any answers that are not NA or No only then we don't have a ctrl
            return 1
    return 0


In [ ]:
# filter through the hearing impairment questions to find the controls
hearing_imp_qs = filtered[hearing_imp_f3393]
f3393_ctrl = hearing_imp_qs.apply(find_ctrl_or_NA, axis=1).to_list()
hearing_imp_qs = filtered[hearing_imp_f2247]
f2247_ctrl = hearing_imp_qs.apply(find_ctrl, axis=1).to_list()
hearing_imp_qs = filtered[hearing_imp_f2257]
f2257_ctrl = hearing_imp_qs.apply(find_ctrl, axis=1).to_list()

In [ ]:
pure_ctrl = [0 if i == 0 and f2247_ctrl[en] == 0 and f2257_ctrl[en] == 0 else 1 for en, i in enumerate(f3393_ctrl)]

In [ ]:
print(len(pure_ctrl) - sum(pure_ctrl), "individuals are controls prior to filtration for icd10, icd9 and self-reported codes")

### 5.2.1. Collect ICD 10 codes to filter out from Ctrl

In [ ]:
exclude_ctrl_icd10 = exclusion[(exclusion["UKBB_field_code"] == "f.41270") & (exclusion["Excluded_from_controls"] == 'Y') ]
exclude_ctrl_icd10

In [ ]:
ex_critia_ctrl_icd10 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_ctrl_icd10["Phenotype"].tolist()]
ex_critia_ctrl_icd10

In [ ]:
icd10 = filtered[icd10_colnames]
icd10

In [ ]:
# collect the individuals who should not be part of controls because of icd 10 codes
ex_fxn_icd10 = lambda row: contains_exclusion(row, ex_critia_ctrl_icd10)
ex_10 = icd10.apply(ex_fxn_icd10, axis=1)

### 5.2.2. Collect ICD 9 codes to filter out from Ctrl

In [ ]:
exclude_ctrl_icd9 = exclusion[(exclusion["UKBB_field_code"] == "f.41271") & (exclusion["Excluded_from_controls"] == 'Y') ]
exclude_ctrl_icd9

In [ ]:
ex_critia_ctrl_icd9 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_ctrl_icd9["Phenotype"].tolist()]
ex_critia_ctrl_icd9

In [ ]:
icd9 = filtered[icd9_colnames]
icd9

In [ ]:
# collect the individuals who should not be part of controls because of icd 9 codes
ex_fxn_icd9 = lambda row: contains_exclusion(row, ex_critia_ctrl_icd9)
ex_9 = icd9.apply(ex_fxn_icd9, axis=1)

### 5.2.3. Collect f20002 codes to filter out from Ctrl

In [ ]:
exclude_ctrl_f20002 = exclusion[(exclusion["UKBB_field_code"] == "f.20002") & (exclusion["Excluded_from_controls"] == 'Y') ]
exclude_ctrl_f20002

In [ ]:
f20002 = filtered[f20002_colnames]
f20002

In [ ]:
ex_critia_ctrl_f20002 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_ctrl_f20002["Phenotype"].tolist()]
ex_critia_ctrl_f20002

In [ ]:
# collect the individuals who should not be part of controls because of self-reported codes
ex_fxn_f20002 = lambda row: contains_exclusion(row, ex_critia_ctrl_f20002)
ex_f20002 = f20002.apply(ex_fxn_f20002, axis=1)

### 5.2.4. Filter out the HI Ctrl

In [ ]:
temp = pure_ctrl | ex_10 | ex_9 | ex_f20002

In [ ]:
# because individuals that are controls are labeled as 0
# temp says True if an individual is not a control and False if it is a control
# ex_10, ex_9, and ex_f20002 are True for individuals that are not controls and False for individuals that are controls
temp = pure_ctrl | ex_10 | ex_9 | ex_f20002

# we set the control as 0 for each individual that is False in temp
filtered_ctrl = [1 if i else 0 for i in temp.to_list()]

In [ ]:
print(len(filtered_ctrl) - sum(filtered_ctrl), "individuals are controls after addition filtration for icd10, icd9 and self-reported codes")

In [ ]:
filtered["hearing_imp_pure_ctrl"] = filtered_ctrl

In [ ]:
filtered

In [ ]:
filtered = filtered.reset_index()

In [ ]:
saved_3_filtered = filtered

In [ ]:
filtered = saved_3_filtered

## 5.3. Identify All Age and Phenotype Columns

In [ ]:
ages_f21003_col = [col for col in filtered if "f.21003" in col]
ages_f21003_col

In [ ]:
ages_f131258_col = [col.strip('"') for col in header if 'f.131258' in col]
ages_f131258_col

In [ ]:
filtered[ages_f21003_col]

In [ ]:
filtered[ages_f131258_col]

In [ ]:
# get the latest time that an individual said no to any of the phenotypes
# return the oldest age that they were
def get_ctrl_age(row):
    phens = [hearing_imp_f3393, hearing_imp_f2247, hearing_imp_f2257]
    ages = []
    if row["hearing_imp_pure_ctrl"] == 0:
        temp_all_ages = row[ages_f21003_col].to_list()
        temp_all_ages.reverse()
        
        for phen in phens:
            temp = row[phen].to_list()
            temp.reverse()
            for en, i in enumerate(temp):
                if not pd.isna(i) and i == "No":
                    ages.append(temp_all_ages[en])
                    break
        ages.sort()
        return ages[-1]
    return pd.NA

# get the earliest time that an individual said yes to having a phenotype
def get_phen_age(row):
    if row[0] == 1:
        temp = row[1:-4].to_list()
        for en, i in enumerate(temp):
            if not pd.isna(i) and i == "Yes":
                return row[ages_f21003_col][en]
    else:
        return pd.NA
    
# return the minium age in the row, or else return NA
def get_min_age(row):
    temp = [i for i in row.to_list() if not pd.isna(i)]
    if len(temp) > 0:
        return min(temp)
    else:
        return pd.NA
    

In [ ]:
# return 1 if we have a yes (used to find phenos)
def find_yes(row):
    for i in row:
        if not pd.isna(i) and i == "Yes":
            return 1
    return 0

# return 1 if we have a match for the mendelian traits and have at least one of the hearing phenotypes
def find_medelian_like(row):
    mendelian_icd10 = ["H903", "H905", "H906", "H908", "H913", "H918", "H919"]
    mendelian_icd9 = ["3891", "3892", "3897", "3898", "3899"]

    if 1 in row[["f3393", "f2247", "f2257"]].to_list():
        for en, i in enumerate(row[icd_10_cols]):
            if not pd.isna(i) and i in mendelian_icd10:
                return 1, get_ages_from_birth(row[icd10_ages[en:en+1]+year_of_birth+month_of_birth])
        for en, i in enumerate(row[icd_9_cols]):
            if not pd.isna(i) and i in mendelian_icd9:
                return 1, get_ages_from_birth(row[icd9_ages[en:en+1]+year_of_birth+month_of_birth])
    return 0, pd.NA

# return a 0 if the individual is not a case and 1 if they are a case
def find_exclusions(row):
    mendelian_icd10 = ["H903", "H904", "H905", "H906", "H907", "H908"]
    mendelian_icd9 = ["3891", "3892", "3897", "3898", "3899"]

    try:
        if 1 == row[0]: # the first column will be one of the phenotypes, "f3393", "f2247", or "f2257"
            for en, i in enumerate(row[icd_10_cols]):
                if not pd.isna(i) and i in mendelian_icd10:
                    if get_ages_from_birth(row[ages_f131258_col+year_of_birth+month_of_birth]) <= 55: # row[1] must be the age of the phenotype
                        return 0, pd.NA
                    return 1, get_ages_from_birth(row[ages_f131258_col+year_of_birth+month_of_birth])
            for en, i in enumerate(row[icd_9_cols]):
                if not pd.isna(i) and i in mendelian_icd9:
                    if get_ages_from_birth(row[icd9_ages[en:en+1]+year_of_birth+month_of_birth]) <= 55:
                        return 0, pd.NA
                    return 1, get_ages_from_birth(row[icd9_ages[en:en+1]+year_of_birth+month_of_birth])
        return int(row[0]), row[1] # the second column should be the ages of that phenotype
    except:
        print(row["name"])


# return 1 if we have a match for the other cases of f3393 or originally had f3393
def find_f3393_other_cases(row):
    icd10 = ["Z461", "Z974"]
    icd9 = ["V412", "V532"]
    if 0 == int(row["f3393"]):
        for en, i in enumerate(row[icd_10_cols]):
            if not pd.isna(i) and i in icd10:
                return 1, get_ages_from_birth(row[icd10_ages[en:en+1]+year_of_birth+month_of_birth])
        for en, i in enumerate(row[icd_9_cols]):
            if not pd.isna(i) and i in icd9:
                return 1, get_ages_from_birth(row[icd9_ages[en:en+1]+year_of_birth+month_of_birth])
    return int(row["f3393"]), row["f3393_age"]

# check if the h919 code exists in the individual
def check_code(row):
    for i in row:
        if not pd.isna(i) and i == "H919":
            return 1
    return 0

In [ ]:
def get_ages_from_birth(row):
    month_dict = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
    year = 0
    month = 0
    i = row[0]
    if not pd.isna(i):
        i = i.split("-")
        year = i[0]
        month = i[1]
    if int(month) >= month_dict[row[month_of_birth[0]]]:
        return int(year) - int(row[year_of_birth[0]])
    return int(year) - int(row[year_of_birth[0]]) - 1

In [ ]:
# f3393
hearing_imp_qs = filtered[hearing_imp_f3393]
filtered["f3393"] = hearing_imp_qs.apply(find_yes, axis=1)
filtered["f3393_age"] = filtered[["f3393"] + hearing_imp_f3393 + ages_f21003_col].apply(get_phen_age, axis=1)
filtered[["f3393", "f3393_age"]] = filtered[["f3393", "f3393_age"] + icd_10_cols + icd_9_cols + icd10_ages + icd9_ages + ages_f131258_col + year_of_birth + month_of_birth].apply(find_exclusions, axis=1, result_type='expand')
filtered[["f3393", "f3393_age"]] = filtered[["f3393", "f3393_age"] + icd_10_cols + icd_9_cols + icd10_ages + icd9_ages + year_of_birth + month_of_birth].apply(find_f3393_other_cases, axis=1, result_type='expand')

In [ ]:
# check if we have f3393 ages
filtered[(filtered["f3393"] == 1) & (pd.isna(filtered["f3393_age"])) & (pd.isna(filtered[ages_f131258_col[0]]))][["f3393", "f3393_age"] + ages_f131258_col]

In [ ]:
# f2247
hearing_imp_qs = filtered[hearing_imp_f2247]
filtered["f2247"] = hearing_imp_qs.apply(find_yes, axis=1)
filtered["f2247_age"] = filtered[["f2247"] + hearing_imp_f2247 + ages_f21003_col].apply(get_phen_age, axis=1)
filtered[["f2247", "f2247_age"]] = filtered[["f2247", "f2247_age"] + icd_10_cols + icd_9_cols + icd10_ages + icd9_ages + ages_f131258_col + year_of_birth + month_of_birth].apply(find_exclusions, axis=1, result_type='expand')

In [ ]:
# check if we have f2247 ages
filtered[(filtered["f2247"] == 1) & (pd.isna(filtered["f2247_age"]))][["f2247", "f2247_age"]]

In [ ]:
# f2257
hearing_imp_qs = filtered[hearing_imp_f2257]
filtered["f2257"] = hearing_imp_qs.apply(find_yes, axis=1)
filtered["f2257_age"] = filtered[["f2257"] + hearing_imp_f2257 + ages_f21003_col].apply(get_phen_age, axis=1)
filtered[["f2257", "f2257_age"]] = filtered[["f2257", "f2257_age"] + icd_10_cols + icd_9_cols + icd10_ages + icd9_ages + ages_f131258_col + year_of_birth + month_of_birth].apply(find_exclusions, axis=1, result_type='expand')

In [ ]:
# check if we have empty f2257 ages
filtered[(filtered["f2257"] == 1) & (pd.isna(filtered["f2257_age"]))][["f2257", "f2257_age"]]

In [ ]:
# mendelian
filtered[["mendelian", "mendelian_age"]] = filtered[icd_10_cols + icd_9_cols + ["f3393", "f2247", "f2257"] + icd10_ages + icd9_ages + year_of_birth + month_of_birth].apply(find_medelian_like, axis=1, result_type='expand')

In [ ]:
# check if we have empty mendelian ages
filtered[(filtered["mendelian"] == 1) & (pd.isna(filtered["mendelian_age"]))][["mendelian", "mendelian_age"]]

In [ ]:
# ctrl age
filtered["ctrl_age"] = filtered[["hearing_imp_pure_ctrl"] + ages_f21003_col +  hearing_imp_f3393 + hearing_imp_f2247 + hearing_imp_f2257].apply(get_ctrl_age, axis=1)

In [ ]:
# check if we have empty ctrl ages
filtered[(filtered["hearing_imp_pure_ctrl"] == 0) & (pd.isna(filtered["ctrl_age"]))][["hearing_imp_pure_ctrl", "ctrl_age"]]

In [ ]:
# f2247_f2257
filtered["f2247_f2257"] = filtered["f2247"] & filtered["f2257"]
filtered["f2247_f2257_age"] = filtered[["f2247_age", "f2257_age"]].apply(get_min_age, axis=1)

In [ ]:
# check if we have empty f2247_f2257 ages
filtered[(filtered["f2247_f2257"] == 1) & (pd.isna(filtered["f2247_f2257_age"]))][["f2247_f2257", "f2247_f2257_age"]]

## 5.4. File Output

In [ ]:
filtered

In [ ]:
filtered[filtered["hearing_imp_pure_ctrl"] == 0][["FID", "IID", "sex", "hearing_imp_pure_ctrl", "ctrl_age", "ethnicity"]].to_csv("pure_ctrl_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["f3393"] == 1][["FID", "IID", "sex", "f3393", "f3393_age", "ethnicity"]]

In [ ]:
filtered[filtered["f3393"] == 1][["FID", "IID", "sex", "f3393", "f3393_age", "ethnicity"]].to_csv("f3393_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["f2247"] == 1][["FID", "IID", "sex", "f2247", "f2247_age", "ethnicity"]]

In [ ]:
filtered[filtered["f2247"] == 1][["FID", "IID", "sex", "f2247", "f2247_age", "ethnicity"]].to_csv("f2247_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["f2257"] == 1][["FID", "IID", "sex", "f2257", "f2257_age", "ethnicity"]]

In [ ]:
filtered[filtered["f2257"] == 1][["FID", "IID", "sex", "f2257", "f2257_age", "ethnicity"]].to_csv("f2257_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["f2247_f2257"] == 1][["FID", "IID", "sex", "f2247_f2257", "f2247_f2257_age", "ethnicity"]]

In [ ]:
filtered[filtered["f2247_f2257"] == 1][["FID", "IID", "sex", "f2247_f2257", "f2247_f2257_age", "ethnicity"]].to_csv("f2247_f2257_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["mendelian"] == 1][["FID", "IID", "sex", "mendelian", "mendelian_age", "ethnicity"]]

In [ ]:
filtered[filtered["mendelian"] == 1][["FID", "IID", "sex", "mendelian", "mendelian_age", "ethnicity"]].to_csv("mendelian_pheno_file.tsv", sep='\t', index=False)

# 6. Tinnitus

## 6.1. Remove inconsistencies or unclear individuals

### 6.1.1. Prior to filtering for inconsistencies

<b>Tinnitus</b> <br>
f.4803 = {'No, never': 76141,
 'Yes, but not now, but have in the past': 11400,
 'Yes, now some of the time': 9788,
 'Yes, now a lot of the time': 2973,
 'Yes, now most or all of the time': 7426,
 'Do not know': 1745,
 'Prefer not to answer': 127}

### 6.1.2. Inconsistencies in the tinnitus answers

In [ ]:
filtered = saved_filtered

In [ ]:
tin_cols = [col for col in filtered if "f.4803" in col]

icd_10_cols = [col for col in df if "f.41270" in col]
icd_9_cols = [col for col in df if "f.41271" in col]

In [ ]:
tin_ans = {"Do not know":9, 'Yes, but not now, but have in the past':1, 'Yes, now some of the time':1, 'Yes, now a lot of the time':1, 'Yes, now most or all of the time':1, 'No, never':0}
options = set()
# pass one pheno at a time
def find_options(row):
    answer = "".join([str(tin_ans[i]) for i in row if not pd.isna(i) and i != "Prefer not to answer"])
    options.add(answer)

In [ ]:
s = filtered[tin_cols].apply(find_options, axis=1)

In [ ]:
options

In [ ]:
# we might have inconsistencies if we have don't knows mixed with other answers or even yes and no together
# options contains the set of all unique codes 

do_not_know_no = [i for i in options if '0' in i and '9' in i and '1' not in i]
do_not_know_yes = [i for i in options if '0' not in i and '9' in i and '1' in i]
yes_no = [i for i in options if '0' in i and '9' not in i and '1' in i]
with_all_three = [i for i in options if '0' in i and '9' in i and '1' in i]

# collecting all possible occurances that can be flagged for being inconsistent
might_inconsistent = do_not_know_no + do_not_know_yes + yes_no + with_all_three

# these are exceptions to the possibilities we have that can be flagged for being inconsistent
exceptions = ["91","911","9111","991","0001","001","0011","01","011","0111", "0091", "091","0911","0991","9001","901","9011", "90", "900", "9000", "990"]

# collect the list of answer combinations that are actually inconsistent by removing the answer combinations that are exceptions
inconsistent = [i for i in might_inconsistent if i not in exceptions]

In [ ]:
inconsistent

#### 6.1.2.1. Filtering out the data

In [ ]:
filtered

In [ ]:
# will return true if that row should be removed
# pass one pheno at a time
def find_inconsistencies(row):
    for i in row:
        if not pd.isna(i) and i == "I am completely deaf":
            return True
    
    answer = "".join([str(tin_ans[i]) for i in row if not pd.isna(i) and i != "I am completely deaf" and i != "Prefer not to answer"])
    if answer in inconsistent:
        return True
    return False

In [ ]:
exclude = filtered[tin_cols].apply(find_inconsistencies, axis=1)
filtered = filtered[~exclude]

In [ ]:
filtered

## 6.2. Identify Pure Control

Need to make sure that for tinnitus we obtain the individuals that say no.

The conditions for saying no are as follows:
* Never saying yes
* Not being inconsistent
* Saying no at least once (prefer not to say is allowed)

We are also not including individuals to be part of the control group if they have certain codes for ICD9, ICD10, or f.20002 (this is in the case that they say no to all tinnitus). However these individuals can still be part of the cases

In [ ]:
tin_ans = {"Do not know":9, 'Yes, but not now, but have in the past':1, 'Yes, now some of the time':1, 'Yes, now a lot of the time':1, 'Yes, now most or all of the time':1, 'No, never':0}

# returns 0 if it's a ctrl or else 1
def find_ctrl(row):
    answer = "".join([str(tin_ans[i]) for i in row if not pd.isna(i) and i != "Prefer not to answer"])
    if "0" in answer and "1" not in answer:
        return False
    return True

In [ ]:
f4803_ctrl = filtered[tin_cols].apply(find_ctrl, axis=1)

In [ ]:
sum(f4803_ctrl)

### 6.2.1. Collect ICD 10 codes to filter out from Ctrl

In [ ]:
exclude_ctrl_icd10 = exclusion[(exclusion["UKBB_field_code"] == "f.41270") & (exclusion["Excluded_from_controls"] == 'Y') ]
exclude_ctrl_icd10

In [ ]:
ex_critia_ctrl_icd10 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_ctrl_icd10["Phenotype"].tolist()]
ex_critia_ctrl_icd10

In [ ]:
icd10 = filtered[icd10_colnames]
icd10

In [ ]:
ex_fxn_icd10 = lambda row: contains_exclusion(row, ex_critia_ctrl_icd10)
ex_10 = icd10.apply(ex_fxn_icd10, axis=1)

### 6.2.2. Collect ICD 9 codes to filter out from Ctrl

In [ ]:
exclude_ctrl_icd9 = exclusion[(exclusion["UKBB_field_code"] == "f.41271") & (exclusion["Excluded_from_controls"] == 'Y') ]
exclude_ctrl_icd9

In [ ]:
ex_critia_ctrl_icd9 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_ctrl_icd9["Phenotype"].tolist()]
ex_critia_ctrl_icd9

In [ ]:
icd9 = filtered[icd9_colnames]
icd9

In [ ]:
ex_fxn_icd9 = lambda row: contains_exclusion(row, ex_critia_ctrl_icd9)
ex_9 = icd9.apply(ex_fxn_icd9, axis=1)

### 6.2.3. Collect f20002 codes to filter out from Ctrl

In [ ]:
exclude_ctrl_f20002 = exclusion[(exclusion["UKBB_field_code"] == "f.20002") & (exclusion["Excluded_from_controls"] == 'Y') ]
exclude_ctrl_f20002

In [ ]:
f20002 = filtered[f20002_colnames]
f20002

In [ ]:
ex_critia_ctrl_f20002 = ["".join( (i.split(" ")[0]).split(".") ) for i in exclude_ctrl_f20002["Phenotype"].tolist()]
ex_critia_ctrl_f20002

In [ ]:
ex_fxn_f20002 = lambda row: contains_exclusion(row, ex_critia_ctrl_f20002)
ex_f20002 = f20002.apply(ex_fxn_f20002, axis=1)

### 6.2.4. Collect individuals with other tinnitus codes to filter out from CTRL

In [ ]:
# check if the given code exists in the individuals
def check_code(row, code):
    for i in row:
        if not pd.isna(i) and i == code:
            return 1
    return 0

In [ ]:
tinn_icd10_check_code = lambda row: check_code(row, "H931")
tinn_icd10 = filtered[icd_10_cols].apply(tinn_icd10_check_code, axis = 1)

In [ ]:
tinn_icd9_check_code = lambda row: check_code(row, "3883")
tinn_icd9 = filtered[icd_9_cols].apply(tinn_icd9_check_code, axis = 1)

In [ ]:
self_report_cols = [col for col in filtered if "f.20002" in col]
tinn_self_report_check_code = lambda row: check_code(row, "1597")
tinn_self_report = filtered[self_report_cols].apply(tinn_self_report_check_code, axis = 1)

### 6.2.5. Filter out Tinnitus Ctrl

In [ ]:
sum(f4803_ctrl | ex_10 | ex_9 | ex_f20002 | tinn_icd10 | tinn_icd9 | tinn_self_report)

In [ ]:
temp = f4803_ctrl | ex_10 | ex_9 | ex_f20002 | tinn_icd10 | tinn_icd9 | tinn_self_report
filtered_ctrl = [1 if i else 0 for i in temp.to_list()]

In [ ]:
filtered["tinnitus_pure_ctrl"] = filtered_ctrl

In [ ]:
filtered

## 6.3. Identify Age

In [ ]:
ages_f21003_col = [col for col in filtered if "f.21003" in col]
ages_f21003_col

In [ ]:
#get the minimum age of each individual in the given columns
def get_min_age(row):
    temp = [i for i in row.to_list() if not pd.isna(i)]
    if len(temp) > 0:
        return min(temp)
    else:
        return pd.NA

In [ ]:
filtered["tinnitus_age"] = filtered[ages_f21003_col].apply(get_min_age, axis=1)

In [ ]:
saved_tinn = filtered

In [ ]:
filtered = saved_tinn

## 6.4. Noisy workplace and Loud Music Variable

Two variables that we need to control for in the analysis are f.4825 (noisy workplace) and f.4836 (loud music).

### 6.4.1. Check for inconsistencies

<b>f.4825 "Have you ever worked in a noisy place where you had to shout to be heard?"</b> <br>
"No"= "0", <br>
"Yes, for less than a year"= "1", <br>
"Yes, for around 1-5 years"= "2", <br>
"Yes, for more than 5 years"= "3", <br>
"Prefer not to answer"= NA, <br>
"Do not know"= NA <br>

<b>f.4836 "Have you ever listened to music for more than 3 hours per week at a volume which you would need to shout to be heard or, if wearing headphones, someone else would need to shout for you to hear them?" </b> <br>
"No"= "0", <br>
"Yes, for less than a year"= "1", <br>
"Yes, for around 1-5 years"= "2", <br>
"Yes, for more than 5 years"= "3", <br>
"Prefer not to answer"= NA, <br>
"Do not know"= NA <br>

In [ ]:
noise_wp_cols = [col for col in df if "f.4825" in col]
loud_music_cols = [col for col in df if "f.4836" in col]

In [ ]:
noise_loud_answers = {"No":0, "Yes, for less than a year":1, "Yes, for around 1-5 years":2, "Yes, for more than 5 years":3}

# if the answers we have are not sorted in the order that they're in the list, then that individual is inconsistent
def find_inconsistencies_noisy_loud(row):
    ans = [noise_loud_answers[i] for i in row if i in noise_loud_answers.keys()] 
    return sorted(ans) != ans
        

In [ ]:
exclude = filtered[noise_wp_cols].apply(find_inconsistencies_noisy_loud, axis=1)
filtered = filtered[~exclude]

In [ ]:
exclude = filtered[loud_music_cols].apply(find_inconsistencies_noisy_loud, axis=1)
filtered = filtered[~exclude]

### 6.4.2. Label Noise and Loud Music

In [ ]:
# find individuals that have yes for either noise or loud sounds
def find_label_noisy_loud(row):
    ans = [noise_loud_answers[i] for i in row if i in noise_loud_answers.keys()]
    if len(ans) > 0:
        return ans[-1]
    return pd.NA

In [ ]:
filtered["noise_wp"] = filtered[noise_wp_cols].apply(find_label_noisy_loud, axis=1)

In [ ]:
filtered["noise_wp"] = filtered["noise_wp"].fillna( int(filtered["noise_wp"].median(skipna=True)) )

In [ ]:
filtered["loud_music"] = filtered[loud_music_cols].apply(find_label_noisy_loud, axis=1)

In [ ]:
filtered["loud_music"] = filtered["loud_music"].fillna( int(filtered["loud_music"].median(skipna=True)) )

In [ ]:
filtered

## 6.5. Identify Cases

**Analysis plan:**

1. Individuals who currently have tinnitus (all four yes categories) vs no never only controlling for sex, age, noisy workplace and loud music frequency ("crude") (No tinnitus vs anytype of tinnitus). For this analysis the missing data of the noise variables was imputed using the median for cases and controls separately

3. Individuals in the two top YES categories vs NO never ('No' tinnitus vs 'Yes, now all of the time' and 'Yes, now most of the time')

4. Individuals that say yes in the top 3 categories vs No never (Remove category 'yes, but not now, but have it in the past')

5. Individuals who currently have tinnitus (all four yes categories and tinnitus codes). Not filtering for issues with noisy workplace and loud music.

In [ ]:
# return 1 if we have a yes (used to find phenos)
def find_yes(row):
    for i in row:
        if not pd.isna(i) and i != "Prefer not to answer" and i in tin_ans.keys() and tin_ans[i] == 1:
            return 1
    return 0

### 6.5.1. Analysis Plan 1

In [ ]:
tin_ans = {"Do not know":9, 'Yes, but not now, but have in the past':1, 'Yes, now some of the time':1, 'Yes, now a lot of the time':1, 'Yes, now most or all of the time':1, 'No, never':0}

In [ ]:
tinn_yes = filtered[tin_cols].apply(find_yes, axis=1)

In [ ]:
filtered["tinnitus_1"] = tinn_yes

### 6.5.2. Analysis Plan 2

In [ ]:
tin_ans = {"Do not know":9, 'Yes, now a lot of the time':1, 'Yes, now most or all of the time':1, 'No, never':0}

In [ ]:
tinn_yes = filtered[tin_cols].apply(find_yes, axis=1)

In [ ]:
filtered["tinnitus_2"] = tinn_yes

### 6.5.3. Analysis Plan 3

In [ ]:
tin_ans = {"Do not know":9, 'Yes, now some of the time':1, 'Yes, now a lot of the time':1, 'Yes, now most or all of the time':1, 'No, never':0}

In [ ]:
tinn_yes = filtered[tin_cols].apply(find_yes, axis=1)

In [ ]:
sum(tinn_yes)

In [ ]:
filtered["tinnitus_3"] = tinn_yes

### 6.5.4. Analysis Plan 4

In [ ]:
filtered_for4 = saved_tinn

In [ ]:
tin_ans = {"Do not know":9, 'Yes, but not now, but have in the past':1, 'Yes, now some of the time':1, 'Yes, now a lot of the time':1, 'Yes, now most or all of the time':1, 'No, never':0}

In [ ]:
tinn_yes = filtered_for4[tin_cols].apply(find_yes, axis=1)

In [ ]:
sum(tinn_yes | tinn_icd10 | tinn_icd9 | tinn_self_report)

In [ ]:
filtered_for4["tinnitus_4"] = tinn_yes | tinn_icd10 | tinn_icd9 | tinn_self_report

In [ ]:
sum(filtered_for4["tinnitus_4"])

## 6.6. File Output

In [ ]:
filtered

In [ ]:
filtered_for4

In [ ]:
filtered[filtered["tinnitus_pure_ctrl"] == 0][["FID", "IID", "sex", "tinnitus_pure_ctrl", "tinnitus_age", "noise_wp", "loud_music"]].to_csv("tinnitus_pure_ctrl_pheno_file.tsv", sep='\t', index=False)
#filtered[filtered["tinnitus_pure_ctrl"] == 0][["FID", "IID", "sex", "tinnitus_pure_ctrl", "ctrl_age"]].to_csv("tinnitus_pure_ctrl_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["tinnitus_1"] == 1][["FID", "IID", "sex", "tinnitus_1", "tinnitus_age", "noise_wp", "loud_music"]].to_csv("tinnitus_1_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["tinnitus_2"] == 1][["FID", "IID", "sex", "tinnitus_2", "tinnitus_age", "noise_wp", "loud_music"]].to_csv("tinnitus_2_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered[filtered["tinnitus_3"] == 1][["FID", "IID", "sex", "tinnitus_3", "tinnitus_age", "noise_wp", "loud_music"]].to_csv("tinnitus_3_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered_for4[filtered_for4["tinnitus_4"] == 1][["FID", "IID", "sex", "tinnitus_4", "tinnitus_age"]].to_csv("tinnitus_4_pheno_file.tsv", sep='\t', index=False)

In [ ]:
filtered_for4[filtered_for4["tinnitus_4"] == 1][["FID", "IID", "sex", "tinnitus_4", "tinnitus_age"]]

# 7. Merge Pheno with Ctrl

## 7.1. f2247, f2257, f3393, and mendilian

In [ ]:
ctrl_file_name = "pure_ctrl_pheno_file.tsv"
f3393_file_name = "f3393_pheno_file.tsv"
f2247_file_name = "f2247_pheno_file.tsv"
f2257_file_name = "f2257_pheno_file.tsv"
f2247_f2257_file_name = "f2247_f2257_pheno_file.tsv"
mendilianlike_file_name = "mendelian_pheno_file.tsv"

In [ ]:
f3393 = pd.read_csv(f3393_file_name, sep="\t")
f2247 = pd.read_csv(f2247_file_name, sep="\t")
f2257 = pd.read_csv(f2257_file_name, sep="\t")
f2247_f2257 = pd.read_csv(f2247_f2257_file_name, sep="\t")
ctrl = pd.read_csv(ctrl_file_name, sep="\t")
mendlike = pd.read_csv(mendilianlike_file_name, sep="\t")

In [ ]:
print("ctrl: ",len(ctrl))
print("f2247: ",len(f2247))
print("f2257: ",len(f2257))
print("f2247_f2257: ",len(f2247_f2257))
print("f3393: ",len(f3393))
print("mendlike: ",len(mendlike))

In [ ]:
mendlike[mendlike["mendelian_age"] > 40]

In [ ]:
mendlike[mendlike["mendelian_age"] < 18]

### 7.1.1. f2247

In [ ]:
# create pheno file
ctrl = ctrl.rename(columns={ctrl.columns[4]:"age", ctrl.columns[3]:f2247.columns[3]})
f2247 = f2247.rename(columns={f2247.columns[4]:"age"})

full_pheno = f2247.append(ctrl)
full_pheno[["FID", "IID", "sex", "f2247", "age"]].to_csv("090321_UKBB_Hearing_difficulty_f2247_expandedwhite_45502cases_96601ctrl", sep='\t', index=False)

In [ ]:
# create PCA pheno file
full_pheno[["FID", "IID", "ethnicity"]].to_csv("090321_UKBB_Hearing_difficulty_f2247_expandedwhite_45502cases_96601ctrl.phenopca", sep='\t', index=False)

In [ ]:
# keep id file for genotype data selection
full_pheno[["FID", "IID"]].to_csv("090321_UKBB_Hearing_difficulty_f2247_expandedwhite_45502cases_96601ctrl.keep_id", sep='\t', index=False, header=False)

### 7.1.2. f2257

In [ ]:
# create pheno file
ctrl = ctrl.rename(columns={ctrl.columns[4]:"age", ctrl.columns[3]:f2257.columns[3]})
f2257 = f2257.rename(columns={f2257.columns[4]:"age"})

full_pheno = f2257.append(ctrl)
full_pheno[["FID", "IID", "sex", "f2257", "age"]].to_csv("090321_UKBB_Hearing_noise_f2257_expandedwhite_65660cases_96601ctrl", sep='\t', index=False)

In [ ]:
# create PCA pheno file
full_pheno[["FID", "IID", "ethnicity"]].to_csv("090321_UKBB_Hearing_noise_f2257_expandedwhite_65660cases_96601ctrl.phenopca", sep='\t', index=False)

In [ ]:
# keep id file for genotype data selection
full_pheno[["FID", "IID"]].to_csv("090321_UKBB_Hearing_noise_f2257_expandedwhite_65660cases_96601ctrl.keep_id", sep='\t', index=False, header=False)

### 7.1.3. f2247_f2257

In [ ]:
# create pheno file
ctrl = ctrl.rename(columns={ctrl.columns[4]:"age", ctrl.columns[3]:f2247_f2257.columns[3]})
f2247_f2257 = f2247_f2257.rename(columns={f2247_f2257.columns[4]:"age"})

full_pheno = f2247_f2257.append(ctrl)
full_pheno[["FID", "IID", "sex", "f2247_f2257", "age"]].to_csv("090321_UKBB_Combined_f2247_f2257_expandedwhite_38410cases_96601ctrl", sep='\t', index=False)

In [ ]:
# create PCA pheno file
full_pheno[["FID", "IID", "ethnicity"]].to_csv("090321_UKBB_Combined_f2247_f2257_expandedwhite_38410cases_96601ctrl.phenopca", sep='\t', index=False)

In [ ]:
# keep id file for genotype data selection
full_pheno[["FID", "IID"]].to_csv("090321_UKBB_Combined_f2247_f2257_expandedwhite_38410cases_96601ctrl.keep_id", sep='\t', index=False, header=False)

### 7.1.4. f3393

In [ ]:
# create pheno file
ctrl = ctrl.rename(columns={ctrl.columns[4]:"age", ctrl.columns[3]:f3393.columns[3]})
f3393 = f3393.rename(columns={f3393.columns[4]:"age"})

full_pheno = f3393.append(ctrl)
full_pheno[["FID", "IID", "sex", "f3393", "age"]].to_csv("090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl", sep='\t', index=False)

In [ ]:
# create PCA pheno file
full_pheno[["FID", "IID", "ethnicity"]].to_csv("090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl.phenopca", sep='\t', index=False)

In [ ]:
# keep id file for genotype data selection
full_pheno[["FID", "IID"]].to_csv("090321_UKBB_Hearing_aid_f3393_expandedwhite_6436cases_96601ctrl.keep_id", sep='\t', index=False, header=False)

### 7.1.5. Mendelian

In [ ]:
# create pheno file
ctrl = ctrl.rename(columns={ctrl.columns[4]:"age", ctrl.columns[3]:mendlike.columns[3]})
mendlike = mendlike.rename(columns={mendlike.columns[4]:"age"})

full_pheno = mendlike.append(ctrl)
full_pheno[["FID", "IID", "sex", "mendelian", "age"]].to_csv("090321_UKBB_Mendelian_expandedwhite_2686cases_96601ctrl", sep='\t', index=False)

In [ ]:
# create PCA pheno file
full_pheno[["FID", "IID", "ethnicity"]].to_csv("090321_UKBB_Mendelian_expandedwhite_2686cases_96601ctrl.phenopca", sep='\t', index=False)

In [ ]:
# keep id file for genotype data selection
full_pheno[["FID", "IID"]].to_csv("090321_UKBB_Mendelian_expandedwhite_2686rcases_96601ctrl.keep_id", sep='\t', index=False, header=False)

## 7.2. Tinnitus

In [ ]:
tinnitus_ctrl_file_name = "tinnitus_pure_ctrl_pheno_file.tsv"
tinnitus_1_file_name = "tinnitus_1_pheno_file.tsv"
tinnitus_2_file_name = "tinnitus_2_pheno_file.tsv"
tinnitus_3_file_name = "tinnitus_3_pheno_file.tsv"
tinnitus_4_file_name = "tinnitus_4_pheno_file.tsv"

In [ ]:
tinnitus_ctrl = pd.read_csv(tinnitus_ctrl_file_name, sep="\t")
tinnitus_1 = pd.read_csv(tinnitus_1_file_name, sep="\t")
tinnitus_2 = pd.read_csv(tinnitus_2_file_name, sep="\t")
tinnitus_3 = pd.read_csv(tinnitus_3_file_name, sep="\t")
tinnitus_4 = pd.read_csv(tinnitus_4_file_name, sep="\t")

In [ ]:
tinnitus_ctrl = tinnitus_ctrl.rename(columns={tinnitus_ctrl.columns[4]:"age"})
tinnitus_1 = tinnitus_1.rename(columns={tinnitus_1.columns[4]:"age"})
tinnitus_2 = tinnitus_2.rename(columns={tinnitus_2.columns[4]:"age"})
tinnitus_3 = tinnitus_3.rename(columns={tinnitus_3.columns[4]:"age"})
tinnitus_4 = tinnitus_4.rename(columns={tinnitus_4.columns[4]:"age"})

In [ ]:
print("ctrl: ",len(tinnitus_ctrl))
print("tinnitus 1: ",len(tinnitus_1))
print("tinnitus 2: ",len(tinnitus_2))
print("tinnitus 3: ",len(tinnitus_3))
print("tinnitus 4: ",len(tinnitus_4))

### 7.2.1. Analysis 1

In [ ]:
tinnitus_ctrl = tinnitus_ctrl.rename(columns={tinnitus_ctrl.columns[3]:tinnitus_1.columns[3]})
full_tinnitus_1 = tinnitus_1.append(tinnitus_ctrl)
full_tinnitus_1

### 7.2.2. Analysis 2

In [ ]:
tinnitus_ctrl = tinnitus_ctrl.rename(columns={tinnitus_ctrl.columns[3]:tinnitus_2.columns[3]})
full_tinnitus_2 = tinnitus_2.append(tinnitus_ctrl)
full_tinnitus_2

### 7.2.3. Analysis 3

In [ ]:
tinnitus_ctrl = tinnitus_ctrl.rename(columns={tinnitus_ctrl.columns[3]:tinnitus_3.columns[3]})
full_tinnitus_3 = tinnitus_3.append(tinnitus_ctrl)
full_tinnitus_3

### 7.2.4. Analysis 4

In [ ]:
tinnitus_ctrl = tinnitus_ctrl.rename(columns={tinnitus_ctrl.columns[3]:tinnitus_4.columns[3]})
full_tinnitus_4 = tinnitus_4.append(tinnitus_ctrl[tinnitus_ctrl.columns[:-2]])
full_tinnitus_4